# Corrector Coil Bmad to Impact-T

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pytao import Tao
from pmd_beamphysics import FieldMesh
import numpy as np
import os

from impact.interfaces.bmad import (
    tao_create_impact_emfield_cartesian_ele,
    tao_create_impact_lattice_and_fieldmaps,
    impact_from_tao,
)
from pmd_beamphysics.fields.corrector_modeling import make_dipole_corrector_fieldmesh


import matplotlib.pyplot as plt

In [ ]:
R = 2 * 2.54e-2  # 2" radius [m]
L = 0.1  # Length along z [m]
theta = np.pi / 2  # Opening angle [rad]
current = 100  # Current [Amp]

FM1 = make_dipole_corrector_fieldmesh(
    current=current,
    xmin=-R,
    xmax=R,
    nx=21,
    ymin=-R,
    ymax=R,
    ny=21,
    zmin=-5 * L / 2,
    zmax=5 * L / 2,
    nz=101,
    mode="saddle",
    R=R,
    L=L,
    theta=theta,
    npts=20,
    plot_wire=True,
    tilt=0,
)
FM2 = make_dipole_corrector_fieldmesh(
    current=current,
    xmin=-R,
    xmax=R,
    nx=21,
    ymin=-R,
    ymax=R,
    ny=21,
    zmin=-5 * L / 2,
    zmax=5 * L / 2,
    nz=101,
    mode="saddle",
    R=R,
    L=L,
    theta=theta,
    npts=20,
    plot_wire=True,
    tilt=np.pi / 2,
)

Write for Bmad

In [ ]:
FM1.write("correctorcoil_horizontal.h5")
FM2.write("correctorcoil_vertical.h5")

In [ ]:
FM1.zmax == -FM1.zmin

# Run Tao

In [ ]:
tao = Tao(lattice_file="lat.bmad", noplot=True)

In [ ]:
IX_ELE = 2

In [ ]:
grid_params = tao.ele_grid_field(IX_ELE, 1, "base")
grid_params

In [ ]:
FM = FieldMesh(grid_params["file"])
FM.plot_onaxis(["Bx", "By"])

In [ ]:
tao_create_impact_emfield_cartesian_ele(
    tao, IX_ELE, file_id=666, output_path=None, cache=None, name=None
)

In [ ]:
tao_create_impact_lattice_and_fieldmaps(tao)

# Tao to Impact-T

Add some "errors":

In [ ]:
tao.cmd("set ele HCOIL tilt = pi/9")
tao.cmd("set ele HCOIL x_offset = -.01")
tao.cmd("set ele HCOIL y_offset = .02")

tao.cmd("set ele VCOIL tilt = pi/3")
tao.cmd("set ele VCOIL x_offset = .03")
tao.cmd("set ele VCOIL y_offset = .04")

In [ ]:
I = impact_from_tao(tao)
I.track1(pz0=10e6)
I.plot("mean_x", ylim=(-0.001, 0.001))

In [ ]:
!cat {I.path}/ImpactT.in

# Compare 

In [ ]:
s1 = tao.lat_list("*", "ele.s")
x1 = tao.lat_list("*", "orbit.vec.1")
y1 = tao.lat_list("*", "orbit.vec.3")

z2 = I.stat("mean_z")
x2 = I.stat("mean_x")
y2 = I.stat("mean_y")

fix, axes = plt.subplots(2)
axes[0].plot(s1, x1 * 1e3, marker="x", label="Bmad")
axes[0].plot(z2, x2 * 1e3, label="Impact-T")
axes[0].legend()
axes[1].plot(s1, y1 * 1e3, marker="*", label="Bmad")
axes[1].plot(z2, y2 * 1e3, label="Impact-T")

plt.legend()
axes[1].set_xlabel(r"$z$ (m)")
axes[0].set_ylabel(r"$x$ (mm)")
axes[1].set_ylabel(r"$y$ (mm)")

# Cleanup

In [ ]:
os.remove("correctorcoil_horizontal.h5")
os.remove("correctorcoil_vertical.h5")